In [ ]:
import sys
sys.path.append('src')

***
## Module Imports

In [ ]:
import nltk
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from nltk.metrics.distance import jaccard_distance
from collections.abc import Iterable
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [ ]:
from data_utils import load_data
from dimension.lexical import *
from dimension.syntactical import *

***
## Data

In [ ]:
train_data, test_data = load_data('data/')
print(
    f"train_data samples: {len(train_data)}, test_data samples: {len(test_data)}"
)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

***
## Similarity functions

In [ ]:
def jaccard_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    return 1 - jaccard_distance(set(s1), set(s2))

In [ ]:
def overlap_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    s1 = set(s1)
    s2 = set(s2)
    intersection = s1.intersection(s2)
    return len(intersection) / min(len(s1), len(s2))

In [ ]:
def cosine_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    s1 = set(s1)
    s2 = set(s2)
    intersection = s1.intersection(s2)
    return len(intersection) / ((len(s1) * len(s2))**2)

In [ ]:
def dice_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    s1 = set(s1)
    s2 = set(s2)
    intersection = s1.intersection(s2)
    return 2 * len(intersection) / (len(s1) + len(s2))

***
## Feature loading

### feature vector builder for dataframe of sentence pairs

Declaration of the function responsible for the iteration over the dataframe containing the sentence pairs (other columns shall be unused). Requires the sentences columns' to be named `"S1"` and `"S2"`.

Returns a numpy array of shape `(n_sentence_pairs, n_features)`

In [ ]:
def get_features(df: pd.DataFrame):
    assert "S1" in df.columns, "S1 not in dataframe"
    assert "S2" in df.columns, "S2 not in dataframe"

    features = [None] * len(df)  # preallocated for memory efficiency

    for index, row in df.iterrows():
        sentence1, sentence2 = row['S1'], row['S2']

        # Get all words
        tokenized_1, tokenized_2 = get_tokenized_sentences(
            sentence1, sentence2, return_unique_words=False)
        tokenized_lc_1, tokenized_lc_2 = get_tokenized_sentences_lowercase(
            tokenized_1, tokenized_2, return_unique_words=False)

        # Get words without stopwords
        no_stopwords_1, no_stopwords_2 = get_tokenized_without_stopwords(
            tokenized_1, tokenized_2, return_unique_words=False)
        no_stopwords_lc_1, no_stopwords_lc_2 = get_tokenized_without_stopwords(
            tokenized_lc_1, tokenized_lc_2, return_unique_words=False)

        # Lemmas
        lemmatized_1, lemmatized_2 = get_lemmas(tokenized_1,
                                                tokenized_2,
                                                return_unique_words=False)
        lemmatized_lc_1, lemmatized_lc_2 = get_lemmas(
            tokenized_lc_1, tokenized_lc_2, return_unique_words=False)

        # Name entities
        sentence_ne_1, sentence_ne_2 = get_named_entities(
            tokenized_1, tokenized_2)

        # Bigrams
        bigrams_1, bigrams_2 = get_ngrams(no_stopwords_1, no_stopwords_2, n=2)
        trigrams_1, trigrams_2 = get_ngrams(no_stopwords_1,
                                            no_stopwords_2,
                                            n=3)

        # Features
        features[index] = [
#             jaccard_similarity(tokenized_1, tokenized_2),
#             jaccard_similarity(tokenized_lc_1, tokenized_lc_2),
#             jaccard_similarity(no_stopwords_1, no_stopwords_2),
#             jaccard_similarity(no_stopwords_lc_1, no_stopwords_lc_2),
#             jaccard_similarity(lemmatized_1, lemmatized_2),
#             jaccard_similarity(lemmatized_lc_1, lemmatized_lc_2),
#             jaccard_similarity(sentence_ne_1, sentence_ne_2),
#             jaccard_similarity(bigrams_1, bigrams_2),
#             jaccard_similarity(trigrams_1, trigrams_2),
#             overlap_similarity(tokenized_1, tokenized_2),
#             overlap_similarity(tokenized_lc_1, tokenized_lc_2),
#             overlap_similarity(no_stopwords_1, no_stopwords_2),
#             overlap_similarity(no_stopwords_lc_1, no_stopwords_lc_2),
#             overlap_similarity(lemmatized_1, lemmatized_2),
#             overlap_similarity(lemmatized_lc_1, lemmatized_lc_2),
#             overlap_similarity(sentence_ne_1, sentence_ne_2),
#             overlap_similarity(bigrams_1, bigrams_2),
#             overlap_similarity(trigrams_1, trigrams_2),
#             cosine_similarity(tokenized_1, tokenized_2),
#             cosine_similarity(tokenized_lc_1, tokenized_lc_2),
#             cosine_similarity(no_stopwords_1, no_stopwords_2),
#             cosine_similarity(no_stopwords_lc_1, no_stopwords_lc_2),
#             cosine_similarity(lemmatized_1, lemmatized_2),
#             cosine_similarity(lemmatized_lc_1, lemmatized_lc_2),
#             cosine_similarity(sentence_ne_1, sentence_ne_2),
#             cosine_similarity(bigrams_1, bigrams_2),
#             cosine_similarity(trigrams_1, trigrams_2),
#             dice_similarity(tokenized_1, tokenized_2),
#             dice_similarity(tokenized_lc_1, tokenized_lc_2),
#             dice_similarity(no_stopwords_1, no_stopwords_2),
            dice_similarity(no_stopwords_lc_1, no_stopwords_lc_2),
#             dice_similarity(lemmatized_1, lemmatized_2),
#             dice_similarity(lemmatized_lc_1, lemmatized_lc_2),
#             dice_similarity(sentence_ne_1, sentence_ne_2),
#             dice_similarity(bigrams_1, bigrams_2),
#             dice_similarity(trigrams_1, trigrams_2)
        ]
    return np.array(features)

In [ ]:
# TEST cell don't delete it =D

first = "My Bonnie White lies over the ocean, in Picadilli Circus at 3:00pm."
second = "My Bonnie lied over the sea! Over the sea..."

tokenized_1, tokenized_2 = get_tokenized_sentences(first,
                                                   second,
                                                   return_unique_words=False)
tokenized_lc_1, tokenized_lc_2 = get_tokenized_sentences_lowercase(
    tokenized_1, tokenized_2, return_unique_words=False)

print(get_lemmas(tokenized_1, tokenized_2))
print(get_lemmas(tokenized_lc_1, tokenized_lc_2))

#TEST cell

### Train features extraction

Using the function declared above, the features are extracted from the `train_data` dataframe. Also the Gold Standard is extracted from its column in the dataframe. The shapes for both numpy vectors are displayed. 

In [ ]:
train_features = get_features(train_data)
train_gs = train_data['Gs'].to_numpy()
print(f"train_features.shape: {train_features.shape}")
print(f"train_gs.shape: {train_gs.shape}")

### Feature scaling

features are scaled using sklearns StandardScaler, where the mean is substracted for each feature and it's divided by the variance of the feature to obtain a unified feature space with zero mean and unit variance.

In [ ]:
scaler = StandardScaler()
scaler.fit(train_features)
train_features_scaled = scaler.transform(train_features)

### SVR Training

In [ ]:
pearson_scorer = make_scorer(lambda y, y_hat: pearsonr(y, y_hat)[0])

gammas = np.logspace(-6, -1, 6)
Cs = np.array([0.5, 1, 2, 4, 8, 10, 15, 20, 50, 100, 200, 375, 500, 1000])
epsilons = np.linspace(0.1, 1, 10)
param = dict(gamma=gammas, C=Cs, epsilon=epsilons)

svr = SVR(kernel='rbf', tol=1)
gssvr = GridSearchCV(svr,
                     param,
                     cv=5,
                     scoring=pearson_scorer,
                     n_jobs=-1,
                     verbose=1)
gssvr = gssvr.fit(train_features_scaled, train_gs)

### Test features extraction

In [ ]:
test_features = get_features(test_data)
test_gs = test_data['Gs'].to_numpy()
print(f"train_features.shape: {test_features.shape}")
print(f"train_gs.shape: {test_gs.shape}")

### Test Inference

In [ ]:
test_features_scaled = scaler.transform(test_features)
predictions = gssvr.predict(test_features_scaled)

### Evaluation metrics

In [ ]:
correlation = pearsonr(predictions, test_gs)[0]

In [ ]:
print('The best 5fold train pearsonr:', gssvr.best_score_)
print(f"Test pearsonr: {correlation}")
print('The best value of gamma:', gssvr.best_estimator_.gamma)
print('The best value of C:', gssvr.best_estimator_.C)
print('The best value of epsilon:', gssvr.best_estimator_.epsilon)

***

# Do not delete ;)

## Recorded Results
|SVR|jaccard|overlap|cosine|dice|jaccard+dice|no_stop_lc_dice|
|---|---|---|---|---|---|---|
|train correlation|0.6402|0.6233|0.3732|0.6397|0.6379|0.6017|
|test_correlation|0.6208|0.6198|0.1056|0.6483|0.6087|0.5905|
|C|10.0|500.0|100.0|375.0|1000.0|2.0|
|gamma|0.01|0.01|0.1|0.0001|0.001|0.1|
|epsilon|0.9|1.0|1.0|0.9|0.2|0.1|